# Final model

In [13]:
import pybamm
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
from scipy.interpolate import interp1d

parameter_values = pybamm.ParameterValues("Chen2020")

# Set model options for SEI degradation
options = {
    "SEI": "reaction limited", 
    "SEI porosity change": "true",
}

# Update the model with the chosen options
model_with_deg = pybamm.lithium_ion.DFN(options)

max_step_size = 120

solver = pybamm.CasadiSolver(mode="safe", max_step_decrease_count=15,dt_max = max_step_size)

num_cycles =   # Number of cycles

pybamm.set_logging_level("NOTICE")

# Experiment from First Snippet
experiment = pybamm.Experiment(
    [
        "Rest for 10 minutes",
        "Charge at 1 C until 4.2 V",
        "Hold at 4.2 V until 50 mA",
        "Rest for 10 minutes",
        "Discharge at 1 C until 2.5 V",
        "Rest for 10 minutes",
    ] * num_cycles,
    period="1 second"
)

# Run First Simulation
deg_sim = pybamm.Simulation(model_with_deg, parameter_values=parameter_values, experiment=experiment, solver=solver)
deg_solution = deg_sim.solve(initial_soc=0.3)
initial_capacity = deg_solution.summary_variables["Capacity [A.h]"][-1]

# Initialize SOH Storage
initial_soc = 0.3
soh_over_cycles = []
all_cycles_data = []


cyc = []


# Secondary Simulation with Dynamic Experiment Parameters
for cycle_N in range(1, num_cycles + 1):

    # deg_solution = deg_sim.solve()


    # Dynamic Parameters
    lower_SoC = 0.3  # Lower State of Charge
    upper_SoC = 0.7  # Upper State of Charge
    C_rate_charge = 1  # C-rate for charging
    C_rate_discharge = 1  # C-rate for discharging

    charging_current = C_rate_charge * current_capacity  # A
    discharging_current = C_rate_discharge * current_capacity  # A
    initial_lower_Q = lower_SoC * current_capacity  # Ah
    initial_upper_Q = upper_SoC * current_capacity  # Ah
    charge_time = (initial_upper_Q - initial_lower_Q) / charging_current  # h
    discharge_time = (initial_upper_Q - initial_lower_Q) / discharging_current  # h

    # Define the experiment for this cycle
    experiment_N_1 = pybamm.Experiment([
        f"Charge at {charging_current} A until 4.2 V",
        "Hold at 4.2 V for 10 minutes",  # Hold step for safety
        f"Discharge at {discharging_current} A until 2.5 V",
        "Hold at 2.5 V for 10 minutes"  # Hold step for safety
        # Add other steps as necessary
    ])

    # Run the simulation for this cycle
    sim = pybamm.Simulation(model_with_deg, parameter_values=parameter_values, experiment=experiment_N_1, solver=solver)
    # Solve the simulation with initial SoC check
    if lower_SoC >= 0 and lower_SoC <= 1:
        if cycle_N == 1:
            deg_solution = sim.solve(initial_soc=lower_SoC)
        else:
            deg_solution = sim.solve(starting_solution=deg_solution)  # Continue from the previous solution
    else:
        raise ValueError("Initial SoC is out of feasible range (0-1)")
    


    # Update Capacity and SOH for each cycle
    current_capacity = deg_solution.summary_variables["Capacity [A.h]"][-1]
    SOH = current_capacity / initial_capacity * 100
    soh_over_cycles.append(SOH)

    # Extract data for this cycle
    times = deg_solution["Time [s]"].entries
    voltage = deg_solution["Terminal voltage [V]"].entries
    current = deg_solution["Current [A]"].entries

    cycle_numbers = [cycle_N] * len(times)

    soh = [soh_over_cycles[cycle_N-1]] * len(times)  # Repeat SOH for each time point

    dt = np.diff(times, prepend=times[0])  # Time differences for integration
    # Initialize SoC array
    soc = np.zeros_like(times)
    soc[0] = initial_soc
    # Calculate SoC at each time step
    for i in range(1, len(times)):
        charge_added = current[i] * dt[i] / 3600  # Convert to Ah
        soc[i] = soc[i-1] + charge_added / current_capacity
        soc[i] = max(0, min(soc[i], 1))  # Ensure SoC stays within 0 and 1

    # Create a DataFrame for this cycle
    cycle_data = pd.DataFrame({
        "Time [s]": times,
        "Cycle": cycle_numbers,
        "Voltage [V]": voltage,
        "Current [A]": current,
        "State of Charge": soc,
        # "Temperature [K]": temperature,
        "SOH": soh
    })
    all_cycles_data.append(cycle_data)

combined_data = pd.DataFrame(cycle_data)
combined_data.to_excel("battery_data.xlsx", index=False)

2023-12-11 15:50:06.543 - [NOTICE] callbacks.on_cycle_start(172): Cycle 1/600 (145.305 us elapsed) --------------------
2023-12-11 15:50:06.544 - [NOTICE] callbacks.on_step_start(180): Cycle 1/600, step 1/1: Rest for 10 minutes
2023-12-11 15:50:07.570 - [NOTICE] callbacks.on_cycle_start(172): Cycle 2/600 (1.027 s elapsed) --------------------
2023-12-11 15:50:07.571 - [NOTICE] callbacks.on_step_start(180): Cycle 2/600, step 1/1: Charge at 1 C until 4.2 V
2023-12-11 15:50:08.995 - [NOTICE] callbacks.on_cycle_start(172): Cycle 3/600 (2.453 s elapsed) --------------------
2023-12-11 15:50:08.997 - [NOTICE] callbacks.on_step_start(180): Cycle 3/600, step 1/1: Hold at 4.2 V until 50 mA
2023-12-11 15:50:12.400 - [NOTICE] callbacks.on_cycle_start(172): Cycle 4/600 (5.858 s elapsed) --------------------
2023-12-11 15:50:12.402 - [NOTICE] callbacks.on_step_start(180): Cycle 4/600, step 1/1: Rest for 10 minutes
2023-12-11 15:50:13.229 - [NOTICE] callbacks.on_cycle_start(172): Cycle 5/600 (6.687 

In [9]:
len(cyc)

5

In [5]:
cyc = []
cycle_N = 3
times = [5,2,3,4]
cycle_numbers = [cycle_N] * len(times)
cyc.append(cycle_numbers)
print(cycle_numbers)
print(cyc)

[3, 3, 3, 3]
[[3, 3, 3, 3]]
